In [90]:
import re


lines = open("input.txt").read()
lines = open("example1.txt").read()


# ############################################################################
# clean the lines
lines = lines.split("\n")
lines[:10]


['...#......',
 '.......#..',
 '#.........',
 '..........',
 '......#...',
 '.#........',
 '.........#',
 '..........',
 '.......#..',
 '#...#.....']

In [91]:
def expand_empty_space(lines):
    lines_copy = lines.copy()
    i = 0
    for line in lines:
        if "#" not in line:
            lines_copy.insert(i, "." * len(line))
            i += 1
        i += 1
    return lines_copy


# Expand empty columns and rows
new_lines = expand_empty_space(lines)

# Transpose the 2D list
new_lines = list(map(list, zip(*new_lines)))
new_lines = expand_empty_space(new_lines)

# Transpose again
new_lines = list(map(list, zip(*new_lines)))


_ = [print("".join(line)) for line in new_lines[:10]]

....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............


In [92]:
from functools import lru_cache

@lru_cache(maxsize=None)
def calculate_distance(coord1, coord2):
    distance = sum([
        abs(coord1[0] - coord2[0]),
        abs(coord1[1] - coord2[1])
    ])
    # print(f"CALCULATE DISTANCE {coord1} --> {coord2} = {distance}")
    return distance

ALL_GALAXY_COORDS = []
for row_num, line in enumerate(new_lines):
    galaxy_coords = [(row_num, i) for i, c in enumerate(line) if c == "#"]
    [ALL_GALAXY_COORDS.append(gc) for gc in galaxy_coords]

print(f"ALL_GALAXY_COORDS: {ALL_GALAXY_COORDS}")

calculate_distance(ALL_GALAXY_COORDS[3 - 1], ALL_GALAXY_COORDS[6 - 1])

ALL_GALAXY_COORDS: [(0, 4), (1, 9), (2, 0), (5, 8), (6, 1), (7, 12), (10, 9), (11, 0), (11, 5)]


17

In [93]:
ALL_GALAXY_COORDS = []
for row_num, line in enumerate(new_lines):
    galaxy_coords = [(row_num, i) for i, c in enumerate(line) if c == "#"]
    [ALL_GALAXY_COORDS.append(gc) for gc in galaxy_coords]

print(ALL_GALAXY_COORDS)

GALAXY_DISTANCES = { }
for i, galaxy_coords in enumerate(ALL_GALAXY_COORDS):
    # ALL_GALAXY_COORDS.remove(galaxy_coords)
    for other_galaxy_coords in ALL_GALAXY_COORDS[i:]:
        dist = calculate_distance(galaxy_coords, other_galaxy_coords)
        cust_key = f"{galaxy_coords} -> {other_galaxy_coords}"
        curr_max_dist = GALAXY_DISTANCES.get(cust_key, 0)
        GALAXY_DISTANCES[cust_key] = max(dist, curr_max_dist)


_ = [print(gc, "-->",  GALAXY_DISTANCES[gc]) for gc in GALAXY_DISTANCES]

sum([GALAXY_DISTANCES[gd] for gd in GALAXY_DISTANCES])

[(0, 4), (1, 9), (2, 0), (5, 8), (6, 1), (7, 12), (10, 9), (11, 0), (11, 5)]
(0, 4) -> (0, 4) --> 0
(0, 4) -> (1, 9) --> 6
(0, 4) -> (2, 0) --> 6
(0, 4) -> (5, 8) --> 9
(0, 4) -> (6, 1) --> 9
(0, 4) -> (7, 12) --> 15
(0, 4) -> (10, 9) --> 15
(0, 4) -> (11, 0) --> 15
(0, 4) -> (11, 5) --> 12
(1, 9) -> (1, 9) --> 0
(1, 9) -> (2, 0) --> 10
(1, 9) -> (5, 8) --> 5
(1, 9) -> (6, 1) --> 13
(1, 9) -> (7, 12) --> 9
(1, 9) -> (10, 9) --> 9
(1, 9) -> (11, 0) --> 19
(1, 9) -> (11, 5) --> 14
(2, 0) -> (2, 0) --> 0
(2, 0) -> (5, 8) --> 11
(2, 0) -> (6, 1) --> 5
(2, 0) -> (7, 12) --> 17
(2, 0) -> (10, 9) --> 17
(2, 0) -> (11, 0) --> 9
(2, 0) -> (11, 5) --> 14
(5, 8) -> (5, 8) --> 0
(5, 8) -> (6, 1) --> 8
(5, 8) -> (7, 12) --> 6
(5, 8) -> (10, 9) --> 6
(5, 8) -> (11, 0) --> 14
(5, 8) -> (11, 5) --> 9
(6, 1) -> (6, 1) --> 0
(6, 1) -> (7, 12) --> 12
(6, 1) -> (10, 9) --> 12
(6, 1) -> (11, 0) --> 6
(6, 1) -> (11, 5) --> 9
(7, 12) -> (7, 12) --> 0
(7, 12) -> (10, 9) --> 6
(7, 12) -> (11, 0) --> 16
(7, 12)

374

Solution: 9.522.407  (9522407)

<br>


---

# Part 2

In [42]:
import re


lines = open("example1.txt").read()
lines = open("input.txt").read()


# ############################################################################
# clean the lines
lines = lines.split("\n")



def get_empty_lines_indexes(lines):
    empty_lines = []
    i = 0
    for i, line in enumerate(lines):
        if "#" not in line:
            empty_lines.append(i)
    return empty_lines


# Expand empty columns and rows
EMPTY_ROWS = get_empty_lines_indexes(lines)

# Transpose the 2D list
transposed = list(map(list, zip(*lines)))
EMPTY_COLS = get_empty_lines_indexes(transposed)


_ = [print(l) for l in lines[:10]]
print(f"EMPTY_ROWS, EMPTY_COLS: {EMPTY_ROWS}, {EMPTY_COLS}")

.....................#..................#..................................................................#...........#....................
................................................................#..................#........................................................
..........................#.....#.............................................................#........#..................................#.
...#.......#...........................................#........................................................#...........................
............................................................................................................................................
........................................................................#.....#.................................................#......#....
...............................................#.................#...................#......#..............#................................
.............

In [48]:
from functools import lru_cache

@lru_cache(maxsize=None)
def calculate_distance(coord1, coord2):
    AUGMENTATION = 1000000 - 1
    distance = sum([
        abs(coord1[0] - coord2[0]),
        abs(coord1[1] - coord2[1])
    ])
    for empty_row in EMPTY_ROWS:
        x1, x2 = min(coord1[0], coord2[0]), max(coord1[0], coord2[0])
        if x1 <= empty_row <= x2:
            distance += AUGMENTATION
    for empty_col in EMPTY_COLS:
        y1, y2 = min(coord1[1], coord2[1]), max(coord1[1], coord2[1])
        if y1 <= empty_col <= y2:
            distance += AUGMENTATION
    return distance


ALL_GALAXY_COORDS = []
for row_num, line in enumerate(lines):
    galaxy_coords = [(row_num, i) for i, c in enumerate(line) if c == "#"]
    [ALL_GALAXY_COORDS.append(gc) for gc in galaxy_coords]



GALAXY_DISTANCES = { }
for i, galaxy_coords in enumerate(ALL_GALAXY_COORDS):
    # ALL_GALAXY_COORDS.remove(galaxy_coords)
    for other_galaxy_coords in ALL_GALAXY_COORDS[i:]:
        dist = calculate_distance(galaxy_coords, other_galaxy_coords)
        cust_key = f"{galaxy_coords} -> {other_galaxy_coords}"
        curr_min_dist = GALAXY_DISTANCES.get(cust_key, 999999999)
        GALAXY_DISTANCES[cust_key] = min(dist, curr_min_dist)



sum([GALAXY_DISTANCES[gd] for gd in GALAXY_DISTANCES])
# 544723432977 

544723432977

Solution: 544.723.432.977 (544723432977)